# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [20]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
  sum += sqrt_value
  print(sum)


1.0
2.414213562373095
4.146264369941973
6.146264369941973
8.382332347441762
10.83182209022494
13.47757340128953
16.30600052603572
19.30600052603572
22.4682781862041
25.7849029765595
29.249004591697254
32.854555867161245


In [19]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
  print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [31]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [28]:
import dlt


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="data_info",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
										table_name="data_info",
										write_disposition="append")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707913700.3332436 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707913700.9672916 is LOADED and contains no failed jobs


In [29]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n data_info table below:")

data_info = conn.sql("SELECT * FROM data_info").df()
display(data_info)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ data_info           │
└─────────────────────┘




 data_info table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707913700.3332436,s8uGcsIUhOABng,None
1,2,Person_2,27,City_A,1707913700.3332436,I3zKtc0asp5D7Q,None
2,3,Person_3,28,City_A,1707913700.3332436,AetckJeL8xu5Gg,None
3,4,Person_4,29,City_A,1707913700.3332436,+XuNjauVjCivsQ,None
4,5,Person_5,30,City_A,1707913700.3332436,D/Y59br9tVRi7Q,None
5,3,Person_3,33,City_B,1707913700.9672916,h/mn/k4E+8M0nQ,Job_3
6,4,Person_4,34,City_B,1707913700.9672916,zzPDCytTZ6aOJg,Job_4
7,5,Person_5,35,City_B,1707913700.9672916,wb3hPPAHsTEzgA,Job_5
8,6,Person_6,36,City_B,1707913700.9672916,MocfJowDzHvu3w,Job_6
9,7,Person_7,37,City_B,1707913700.9672916,6Vm9ErRsX6JWWA,Job_7


In [30]:
sum_ages = conn.sql("SELECT SUM(age) FROM data_info").df()
display(sum_ages)


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [32]:
import dlt


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="data_info",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
										table_name="data_info",
										write_disposition="merge",
                    primary_key='ID')

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707914233.8006845 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707914234.4252083 is LOADED and contains no failed jobs


In [33]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n data_info table below:")

data_info = conn.sql("SELECT * FROM data_info").df()
display(data_info)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ data_info           │
└─────────────────────┘




 data_info table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707914233.8006845,n5ma2zYjmIjycg,None
1,2,Person_2,27,City_A,1707914233.8006845,9zxlc9MJxOX3Og,None
2,8,Person_8,38,City_B,1707914234.4252083,eP1Vtda33iMHTQ,Job_8
3,7,Person_7,37,City_B,1707914234.4252083,Mr6XRkLM/x9Lag,Job_7
4,5,Person_5,35,City_B,1707914234.4252083,lp6MiGBG5tn5dA,Job_5
5,4,Person_4,34,City_B,1707914234.4252083,FuH3ft7B8FsucQ,Job_4
6,3,Person_3,33,City_B,1707914234.4252083,XxvIXEeA5VdERg,Job_3
7,6,Person_6,36,City_B,1707914234.4252083,qCVeCFfypBDqSQ,Job_6


In [34]:
sum_ages = conn.sql("SELECT SUM(age) FROM data_info").df()
display(sum_ages)


,sum(age)
0,266.0


# Solution: First make sure that the following modules are installed:

In [21]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX